# Lesson 7 notes

### Check lesson7-resnet-mnist.ipynb
[Jump to lesson 7 on resnet-mnist](https://youtu.be/9spwoDYwW_I?t=127)

### Datablock - ImageItemList - from_folder()
```
il = ImageList.from_folder(path, convert_mode='L')
```

convert_mode='L' is passed to the library pillow, and open the image as black and white

## defaults.cmap

set the default color map for fastai

## split_by_folder
```
sd = il.split_by_folder(train='training', valid='testing')
```
you can use .no_split() if you don't want validation set.

cannot skip this method.

you can give the folder name on ='name' attribute.

## Transform on handwriting MNIST

we cannot flip it, cannot rotate, it will change the meaning.

we cannot zoom, it will be too fuzzy on this image size.

so just do padding.
```
tfms = ([*rand_pad(padding=3, size=28, mode='zeros')], [])
```
Also, no transform on validation set ( see [] )

## Number of channel in CNN
```
conv(1, 8) # 14
```
why 8 channel? you pick.

also, 14 is the size of the image on the layer, since it is stride=2

## Flatten()

The last of conv(16, 10) gives out a 3D tensor of 10 by 1 by 1.
But the loss function only take array ( array of 10 elements of popubilities for each digits ).

Flatten() remove those extra dimentions

### Pytorch module as function
```
model(xb)
```
We can use any pytorch.nn.module and called it as function, it will call forward() method for you. It is by design.

## Stride-2 vs Stride-1

Stride-1 does not change output activations size, stride-2 reduces by half. So you can add as many stride-1 layer as you like.

But it is not as simple. There are below problems

## Kaiming He paper
![](lesson7/lesson7-1.png)
Let's look at the training error of a network of 20 layers, and one with 56 layers.

56 layers one have a lot more stride-1 convolutions in the middle. So the one with more parameters should seriously overfit right? So you would expect the 56 layer one to zip down to zero-ish training error pretty quickly.

But this is not happen, it is actually worse.

## Kaiming He assumtion

He did not know why this problem occur, but he said if he can change the 56 layer a little bit, he can make it as good as the 20 layers one.

![](lesson7/lesson7-2.png)

For every 2 layer, instead of the normal output = c2(c1(x)), He change it to output = x + c2(c1(x)).

In other words, if the weights is zero, the layers becomes nothing happen at all, skip connection ( no change, pass-on ). That's why it is called identity x.


This is called a res-block. and the result is revolutionary.

## Why better result?
![](lesson7/lesson7-3.png)

The x and y is the weights' space, while z is the loss. without res-block, it is very bumpy.

## DenseNet
![](lesson7/lesson7-4.png)
So basically, instaed of x + c2(c1(x)), we concatenate it.

Since the dense-block is keeping the original pixel, it works well on segmentation. Because you want to reconstruct the original pixel of the picture

But going deeper every layer, since we concat it, the ouput get bigger in size. so it is very memory intensive.

TODO

## Segmentation

check lesson3-camvid-tiramisu.ipynb

### Fastai SegmentationItemList

It will automatically shows the colorcoded pixel on top of the image, where different color represent a class of object that its classified.

### How Segmentation works

In order for the model to classify a particular pixel to be a pedestrian, while another pixel is a bicyclist, It needs to know what a pedestrian looks like, in a broader sense. It needs to understand that this area of pixel belongs to a pedestrian.

It needs to really understand a lot about this picture before classifying each pixel.

We can get it work well by pre-training.

## U-Net
[Jump to lesson7 U-net lecture](https://youtu.be/9spwoDYwW_I?t=1860)
![](lesson7/lesson7-5.png)

It basically have 3 part. First part ( left side of U ) is to use maybe stride-2 conv to half-size the activations, while creating more channel ( remember, each face plane is a feature ).

Seoncd part ( bottom of U ) is to stop that stride-2 conv when the activation image hit a certain size ( let say 11 by 11 ). Don't go down to 1 by 1, this is not a classifier.

Thrid part ( right side of U ) is to re-construct a image by increasing the activation image size. We can do it by doing stride-half convolution, also known as a deconvolution, or transposed convolution.

## Stride-half Convolution - slow method
![](lesson7/lesson7-6.png)
The blue are inputs ( 2 by 2 ). It is padded to a imaginary 7 by 7 plane, as shown.

The shadow is the 3 by 3 kernal.

Performing image kernal computation, we got a 5 by 5 output ( the green part ).

But, it is very slow, a lot of computation. 

Also, the red circle part, it has a lot of padded zero, 8 or 7 out of 9 in the shadow is zero. It waste a lot of information.

## Stride-half Convolution - better method
![](lesson7/lesson7-7.png)
Instead of padding, just up-sample it.

It is called nearest neighbor interpolation.

Another simular method called bilinear interpolation. It is kind of like a weighted average for each cell.

## Copy and crop
![](lesson7/lesson7-8.png)
Even though we can use stride-half convolution, it is still impossible to get a image activation of 28 by 28, get generate back to 572 by 572, with that fine-details ( the color-coded the shape of pedestrian so well ).

There are also path ( copy and crop ), from each down-sampling on left, connected to right up-sampling. Think of it kind of like skip-connection in res-block.

Those copy and crop is concatenated into the up-sampling layer ( show as blue-framed white box )

### Check lecture on UNet fastai source code explanation
[Jump to Lecture on Unet source code](https://youtu.be/9spwoDYwW_I?t=2682)